In [28]:
import sqlglot
import pandas as pd
import numpy as np
from sqlglot import Dialect
from sqlglot.expressions import Func
from sqlglot.generator import Generator
from sqlglot.parser import Parser
from sklearn.cluster import KMeans
from ml import MachineLearning

# Definindo a expressão CLUSTER()
class Cluster(Func):
    arg_count = 1  # Aceita apenas 1 argumento (coluna)

    def execute(self, df, column_name):
        if column_name not in df.columns:
            raise ValueError(f"A coluna '{column_name}' não foi encontrada no DataFrame.")

        data = df[column_name].values.reshape(-1, 1)
        mlclass = MachineLearning()

        kmeans = mlclass.cluster_kmeans(data, n_clusters=3)
        # kmeans = KMeans(n_clusters=3, random_state=42, n_init="auto")
        # clusters = kmeans.fit_predict(data)

        # Adicionando a coluna de cluster ao DataFrame
        df["cluster"] = kmeans
        return df

    def to_cte(self, df, alias="resultados_cluster"):
        """
        Retorna um DataFrame representando a CTE gerada pelo CLUSTER.

        :param df: DataFrame contendo os dados.
        :param alias: Nome da CTE.
        :return: DataFrame transformado.
        """
        return self.execute(df, self.this.name)


# Definindo a expressão CLUSTER()
class Scatterplot(Func):
    arg_count = 1  # Aceita apenas 1 argumento (coluna)

    def execute(self, df, column_name):
        print("hi")
        return True
    
    def to_cte(self, df, alias="resultados_cluster"):
        """
        Retorna um DataFrame representando a CTE gerada pelo CLUSTER.

        :param df: DataFrame contendo os dados.
        :param alias: Nome da CTE.
        :return: DataFrame transformado.
        """
        return self.execute(df, self.this.name)



# Criando um dialeto personalizado para reconhecer CLUSTER()
class CustomDialect(Dialect):
    class Parser(Parser):
        FUNCTIONS = {**Parser.FUNCTIONS, "CLUSTER": Cluster.from_arg_list, "SCATTER": Scatterplot.from_arg_list}

    class Generator(Generator):
        TRANSFORMS = {
            **Generator.TRANSFORMS,
            Cluster: lambda self, expr: f"CLUSTER({self.sql(expr.this)})",
            Scatterplot: lambda self, expr: f"SCATTER({self.sql(expr.this)})",
        }


# Simulando um DataFrame de entrada
df = pd.DataFrame({
    "id": range(1, 10),
    "price": [10, 20, 30, 200, 220, 250, 1000, 1100, 1200],
})

# Query SQL simulada
query = """
WITH resultados_cluster AS (SELECT CLUSTER(price) FROM df)  
SELECT * FROM resultados_cluster VISUALIZE AS SCATTER(*);
"""

# Parseando a query
expression = sqlglot.parse_one(query, dialect=CustomDialect)

# Encontrar a expressão CLUSTER na AST
cluster_expr = expression.find(Cluster)

# Executar a função CLUSTER e gerar um DataFrame com os resultados
df_clusterizado = cluster_expr.to_cte(df)

vis_exp = expression.find(Scatterplot)
vis_exp.to_cte(df)

# from visualizations import Visualization

# vis = Visualization()

# vis.scatter(df_clusterizado['id'],df_clusterizado['cluster'])

# Exibir o DataFrame final com os clusters
print(df_clusterizado)


ParseError: Invalid expression / Unexpected token. Line 3, Col: 45.
   resultados_cluster AS (SELECT CLUSTER(price) FROM df)  
SELECT * FROM resultados_cluster VISUALIZE [4mAS[0m SCATTER(*);


In [32]:
import sqlglot
import pandas as pd
import numpy as np
from sqlglot import Dialect
from sqlglot.expressions import Func
from sqlglot.generator import Generator
from sqlglot.parser import Parser
from sklearn.cluster import KMeans
from ml import MachineLearning
from sqlglot.tokens import Tokenizer

# Definindo a expressão CLUSTER()
class Cluster(Func):
    arg_count = 1  # Aceita apenas 1 argumento (coluna)

    def execute(self, df, column_name):
        if column_name not in df.columns:
            raise ValueError(f"A coluna '{column_name}' não foi encontrada no DataFrame.")

        data = df[column_name].values.reshape(-1, 1)
        # TODO fazer rodar com a minha classe do Kmenas
        
        mlclass = MachineLearning()

        kmeans = mlclass.cluster_kmeans(data, n_clusters=3)
        # kmeans = KMeans(n_clusters=3, random_state=42, n_init="auto")
        # clusters = kmeans.fit_predict(data)

        # Adicionando a coluna de cluster ao DataFrame
        df["cluster"] = kmeans
        return df

    def to_cte(self, df, alias="resultados_cluster"):
        """
        Retorna um DataFrame representando a CTE gerada pelo CLUSTER.

        :param df: DataFrame contendo os dados.
        :param alias: Nome da CTE.
        :return: DataFrame transformado.
        """
        return self.execute(df, self.this.name)

class Scatter(Func):
    arg_count = 1  # Aceita apenas 1 argumento (coluna)

    def execute(self, df, column_name):
        print("hi")
    def to_cte(self, df, alias="resultados_cluster"):
        """
        Retorna um DataFrame representando a CTE gerada pelo CLUSTER.

        :param df: DataFrame contendo os dados.
        :param alias: Nome da CTE.
        :return: DataFrame transformado.
        """
        return self.execute(df, self.this.name)


# Criando um dialeto personalizado para reconhecer CLUSTER()
class CustomDialect(Dialect):
    class Tokenizer(Tokenizer):
        KEYWORDS = {
            **Tokenizer.KEYWORDS,
            "VISUALIZE": "VISUALIZE",
            "lineplot": "LINEPLOT",
        }

    class Parser(Parser):
        FUNCTIONS = {**Parser.FUNCTIONS, "CLUSTER": Cluster.from_arg_list, "SCATTER": Scatter.from_arg_list}

        # def _parse_visualize(self):
        #     self._match_texts(["VISUALIZE"])  # Reconhece "VISUALIZE"
        #     expression = self._parse_function()  # Captura scatter(cluster), lineplot(x, y)
        #     return self.expression(Scatter, this=expression)
        
        # STATEMENT_PARSERS = {**Parser.STATEMENT_PARSERS, "VISUALIZE": _parse_visualize}

    class Generator(Generator):
        TRANSFORMS = {
            **Generator.TRANSFORMS,
            Cluster: lambda self, expr: f"CLUSTER({self.sql(expr.this)})",
            Scatter: lambda self, expr: f"SCATTER({self.sql(expr.this)})",
        }

df = pd.DataFrame({
    "id": range(1, 10),
    "price": [10, 20, 30, 200, 220, 250, 1000, 1100, 1200],
})

query = """
WITH resultados_cluster AS (SELECT CLUSTER(price) FROM df)  
SELECT SCATTER(price) FROM resultados_cluster;
"""

expression = sqlglot.parse_one(query, dialect=CustomDialect)

cluster_expr = expression.find(Cluster)

df_clusterizado = cluster_expr.to_cte(df)

print(df_clusterizado)


   id  price  cluster
0   1     10        2
1   2     20        2
2   3     30        2
3   4    200        1
4   5    220        1
5   6    250        1
6   7   1000        0
7   8   1100        0
8   9   1200        0


# Integrando o Visualize do mesmo jeito que o cluster

E aproveitando pra testar o esquema de rodar tudo que ele encontrar na arvore

In [6]:
import sqlglot
import pandas as pd
import numpy as np
from sqlglot import Dialect
from sqlglot.expressions import Func
from sqlglot.generator import Generator
from sqlglot.parser import Parser
from sqlglot.tokens import Tokenizer, TokenType
from sklearn.cluster import KMeans
from ml import MachineLearning

# Definindo a expressão CLUSTER()
class Cluster(Func):
    arg_count = 1  # Aceita apenas 1 argumento (coluna)

    def execute(self, df, column_name):
        if column_name not in df.columns:
            raise ValueError(f"A coluna '{column_name}' não foi encontrada no DataFrame.")

        data = df[column_name].values.reshape(-1, 1)
        # TODO fazer rodar com a minha classe do Kmenas
        
        mlclass = MachineLearning()

        kmeans = mlclass.cluster_kmeans(data, n_clusters=3)
        # kmeans = KMeans(n_clusters=3, random_state=42, n_init="auto")
        # clusters = kmeans.fit_predict(data)

        # Adicionando a coluna de cluster ao DataFrame
        df["cluster"] = kmeans
        return df

    def to_cte(self, df, alias="resultados_cluster"):
        """
        Retorna um DataFrame representando a CTE gerada pelo CLUSTER.

        :param df: DataFrame contendo os dados.
        :param alias: Nome da CTE.
        :return: DataFrame transformado.
        """
        return self.execute(df, self.this.name)

class Visualize(Func):
    # arg_count = 1  # Aceita apenas 1 argumento (coluna)
    arg_types = {"this": True}  # O que vem depois de VISUALIZE AS
    print("Func:")
    print(Func)


    def execute(self, df, visualization_expr):
        print(f"Executando visualização para: {visualization_expr}")
        
        return True  # Aqui você pode chamar a classe de visualização real

    def to_cte(self, df, alias="visualizacao_resultados"):
        return self.execute(df, self.this)


    # def execute(self, df, column_name):
    #     if column_name not in df.columns:
    #         raise ValueError(f"A coluna '{column_name}' não foi encontrada no DataFrame.")

    #     print(f"Column name: {column_name}")

    #     data = df[column_name].values.reshape(-1, 1)
    #     # TODO fazer rodar com a minha classe do Kmenas
        
    #     visuzalization = Visualization()
    #     visuzalization.lineplot(data,[data])
    #     return True

    # def to_cte(self, df, alias="resultados_cluster"):
    #     """
    #     Retorna um DataFrame representando a CTE gerada pelo CLUSTER.

    #     :param df: DataFrame contendo os dados.
    #     :param alias: Nome da CTE.
    #     :return: DataFrame transformado.
    #     """
    #     print(f"self this name: {self.this.name}")


    #     return self.execute(df, self.this.name)



# Criando um dialeto personalizado para reconhecer CLUSTER()
class CustomDialect(Dialect):
    class Tokenizer(Tokenizer):
        # Definindo CLUSTER como uma palavra-chave especial
        KEYWORDS = {**Tokenizer.KEYWORDS, "CLUSTER": "CLUSTER", "VISUALIZE": "VISUALIZE", "scatter":"SCATTER"}

    class Parser(Parser):
        FUNCTIONS = {**Parser.FUNCTIONS, "CLUSTER": Cluster.from_arg_list, "VISUALIZE": Visualize.from_arg_list, }

        def _parse_visualize(self):
            self._match_texts(["VISUALIZE"])  # Reconhece "VISUALIZE"
            expression = self._parse_function()  # Captura scatter(cluster), lineplot(x, y)
            return self.expression(Visualize, this=expression)

        STATEMENT_PARSERS = {**Parser.STATEMENT_PARSERS, "VISUALIZE": _parse_visualize}



    class Generator(Generator):
        TRANSFORMS = {
            **Generator.TRANSFORMS,
            Cluster: lambda self, expr: f"CLUSTER({self.sql(expr.this)})",
            Visualize: lambda self, expr: f"VISUALIZE {self.sql(expr.this)}",
        }


# Simulando um DataFrame de entrada
df = pd.DataFrame({
    "id": range(1, 10),
    "price": [10, 20, 30, 200, 220, 250, 1000, 1100, 1200],
})

# Query SQL simulada
query = """
WITH resultados_cluster AS (SELECT CLUSTER(price) FROM df)  
SELECT * FROM resultados_cluster VISUALIZE scatter(cluster);
"""

# Parseando a query
expression = sqlglot.parse_one(query, dialect=CustomDialect)
# parsed_query = sqlglot.parse_one(query)

# expression
# Encontrar a expressão CLUSTER na AST
cluster_expr = expression.find(Cluster)

# Executar a função CLUSTER e gerar um DataFrame com os resultados
df_clusterizado = cluster_expr.to_cte(df)

vis_exp = expression.find(Visualize)

# Executar a função CLUSTER e gerar um DataFrame com os resultados
vis_exp.to_cte(df_clusterizado)

# Exibir o DataFrame final com os clusters
# print(df_clusterizado)


Func:
<class 'sqlglot.expressions.Func'>


ParseError: Expecting ). Line 2, Col: 42.
  
WITH resultados_cluster AS (SELECT [4mCLUSTER[0m(price) FROM df)  
SELECT * FROM resultados_cluster VISUALIZE scatter(cluster);


In [16]:
import sqlglot
from sqlglot.dialects.dialect import Dialect
from sqlglot.parser import Parser
from sqlglot.tokens import Tokenizer, TokenType
from sqlglot.generator import Generator
import sqlglot.expressions as exp
from sqlglot.expressions import Func
from ml import MachineLearning
import pandas as pd
import numpy as np

class Cluster(Func):
    arg_count = 1  # Aceita apenas 1 argumento (coluna)

    def execute(self, df, column_name):
        if column_name not in df.columns:
            raise ValueError(f"A coluna '{column_name}' não foi encontrada no DataFrame.")

        data = df[column_name].values.reshape(-1, 1)
        # TODO fazer rodar com a minha classe do Kmenas
        
        mlclass = MachineLearning()

        kmeans = mlclass.cluster_kmeans(data, n_clusters=3)
        # kmeans = KMeans(n_clusters=3, random_state=42, n_init="auto")
        # clusters = kmeans.fit_predict(data)

        # Adicionando a coluna de cluster ao DataFrame
        df["cluster"] = kmeans
        return df

    def to_cte(self, df, alias="resultados_cluster"):
        """
        Retorna um DataFrame representando a CTE gerada pelo CLUSTER.

        :param df: DataFrame contendo os dados.
        :param alias: Nome da CTE.
        :return: DataFrame transformado.
        """
        return self.execute(df, self.this.name)

# Definição da classe Visualize
class Visualize(Func):
    arg_types = {"this": True}  # Aceita a expressão após VISUALIZE AS

    def execute(self, df, visualization_expr):
        print(f"Executando visualização para: {visualization_expr}")
        return True  # Aqui você pode chamar a classe de visualização real

    def to_cte(self, df, alias="visualizacao_resultados"):
        return self.execute(df, self.this)

# Criando um dialeto personalizado
class CustomDialect(Dialect):
    class Tokenizer(Tokenizer):
        KEYWORDS = {
            **Tokenizer.KEYWORDS,
            "CLUSTER":"CLUSTER",
            "VISUALIZE": "VISUALIZE",
            "scatter": "SCATTER",
            "lineplot": "LINEPLOT",
        }

    class Parser(Parser):

        FUNCTIONS = {
            **Parser.FUNCTIONS,
            "CLUSTER": Cluster.from_arg_list,
            "VISUALIZE": Visualize.from_arg_list, 
        }

        # def _parse_visualize(self):
        #     """Faz o parsing de VISUALIZE AS <expressão>"""
        #     self._match_texts(["VISUALIZE"])  # Reconhece "VISUALIZE"
        #     self._match_texts(["AS"])         # Reconhece "AS"
        #     expression = self._parse_function()  # Captura scatter(cluster), lineplot(x, y)
        #     return self.expression(Visualize, this=expression)
        
        # STATEMENT_PARSERS = {**Parser.STATEMENT_PARSERS, "VISUALIZE": _parse_visualize, "CLUSTER":_parse_cluster}

    class Generator(Generator):
        TRANSFORMS = {
            **Generator.TRANSFORMS,
            Cluster: lambda self, expr: f"CLUSTER({self.sql(expr.this)})",
            Visualize: lambda self, expr: f"VISUALIZE AS {self.sql(expr.this)}",
        }

# Simulando um DataFrame de entrada
df = pd.DataFrame({
    "id": range(1, 10),
    "price": [10, 20, 30, 200, 220, 250, 1000, 1100, 1200],
})

# Query SQL simulada
query = """
WITH resultados_cluster AS (SELECT CLUSTER(price) FROM df)  
SELECT * FROM resultados_cluster;
"""

# Parseando a query
expression = sqlglot.parse_one(query, dialect=CustomDialect)

# Encontrar a expressão CLUSTER na AST
cluster_expr = expression.find(Cluster)

# Executar a função CLUSTER e gerar um DataFrame com os resultados
df_clusterizado = cluster_expr.to_cte(df)

print(df_clusterizado)



ParseError: Expecting ). Line 2, Col: 42.
  
WITH resultados_cluster AS (SELECT [4mCLUSTER[0m(price) FROM df)  
SELECT * FROM resultados_cluster;


In [ ]:
import sqlglot
import pandas as pd
from sqlglot import Dialect
from sqlglot.expressions import Func, Expression
from sqlglot.generator import Generator
from sqlglot.parser import Parser
from sqlglot.tokens import Tokenizer, TokenType
from ml import MachineLearning
from sqlglot.expressions import Command

# Testar como criar meu próprio TokenType

# Definindo a expressão CLUSTER()
class Cluster(Func):
    arg_count = 1

    def execute(self, df, column_name):
        if column_name not in df.columns:
            raise ValueError(f"A coluna '{column_name}' não foi encontrada no DataFrame.")

        mlclass = MachineLearning()
        data = df[column_name].values.reshape(-1, 1)
        kmeans = mlclass.cluster_kmeans(data, n_clusters=3)
        
        df["cluster"] = kmeans
        return df

    def to_cte(self, df, alias="resultados_cluster"):
        return self.execute(df, self.this.name)


# Definindo a expressão SCATTER()
class Scatterplot(Func):
    arg_count = 1

    def execute(self, df, column_name):
        print(f"Gerando scatter plot para {column_name}")
        return True

    def to_cte(self, df, alias="visualizacao_scatter"):
        return self.execute(df, self.this.name)


# Criando uma expressão VISUALIZE para reconhecer "VISUALIZE AS SCATTER()"
class Visualize(Expression):
    arg_types = {"this": True}  # Aceita um argumento (o tipo de visualização)
    print("hello")
    def execute(self):
        print(f"Executando VIS")
        print(self)

        return True

    def to_cte(self):
        return self.execute()


# Criando um dialeto personalizado
class CustomDialect(Dialect):
    class Tokenizer(Tokenizer):
        """Adiciona palavras-chave personalizadas"""
        
        KEYWORDS = {
            **Tokenizer.KEYWORDS,
            "CLUSTER": TokenType.FUNCTION,
            "SCATTER": TokenType.FUNCTION,
            "VISUALIZE": TokenType.COMMAND
        }

    class Parser(Parser):
        FUNCTIONS = {
            **Parser.FUNCTIONS,
            "CLUSTER": Cluster.from_arg_list,
            "SCATTER": Scatterplot.from_arg_list,
        }

        def _parse_visualize(self):
            """Faz parsing de VISUALIZE AS SCATTER(price)"""
            self._match_texts(["VISUALIZE"])  # Reconhece "VISUALIZE"
            function_expr = self._parse_function()  # Captura SCATTER(price)
            return self.expression(Visualize, this=function_expr)

        # Adiciona ao parser principal para capturar VISUALIZE AS
        STATEMENT_PARSERS = {
            **Parser.STATEMENT_PARSERS,
            "VISUALIZE": _parse_visualize,
        }

    class Generator(Generator):
        TRANSFORMS = {
            **Generator.TRANSFORMS,
            Cluster: lambda self, expr: f"CLUSTER({self.sql(expr.this)})",
            Scatterplot: lambda self, expr: f"SCATTER({self.sql(expr.this)})",
            Visualize: lambda self, expr: f"VISUALIZE AS {self.sql(expr.this)}",
        }


# Simulando um DataFrame de entrada
df = pd.DataFrame({
    "id": range(1, 10),
    "price": [10, 20, 30, 200, 220, 250, 1000, 1100, 1200],
})

# Query SQL simulada
query = """
WITH resultados_cluster AS (SELECT CLUSTER(price))  
SELECT * FROM resultados_cluster;  
VISUALIZE SCATTER(price);
"""

# Parseando a query
expression_list = sqlglot.parse(query, dialect=CustomDialect)
print(expression_list)



hello


ParseError: Required keyword: 'this' missing for <class '__main__.Visualize'>. Line 4, Col: 17.
  
WITH resultados_cluster AS (SELECT CLUSTER(price))  
SELECT * FROM resultados_cluster;  
VISUALIZE [4mSCATTER[0m(price);


In [27]:
import sqlglot
import pandas as pd
from sqlglot import Dialect
from sqlglot.expressions import Func, Expression, Command
from sqlglot.generator import Generator
from sqlglot.parser import Parser
from sqlglot.tokens import Tokenizer, TokenType
from ml import MachineLearning

class Cluster(Func):
    arg_count = 1

    def execute(self, df, column_name):
        if column_name not in df.columns:
            raise ValueError(f"A coluna '{column_name}' não foi encontrada no DataFrame.")
        print("Hi ML")
        mlclass = MachineLearning()
        data = df[column_name].values.reshape(-1, 1)
        kmeans = mlclass.cluster_kmeans(data, n_clusters=3)
        
        df["cluster"] = kmeans
        print(df)
        return df

    def to_cte(self, df, alias="resultados_cluster"):
        print(df)
        return self.execute(df, self.this.name)

class Scatterplot(Func):
    arg_count = 1

    def execute(self, df, column_name):
        print(f"Gerando scatter plot para {column_name}")
        return True

    def to_cte(self, df, alias="visualizacao_scatter"):
        return self.execute(df, self.this.name)

class Visualize(Expression):
    arg_types = {"this": True}

    def execute(self, df):
        print("Executando VISUALIZE")
        return self.this.execute(df, self.this.this.name)

    def to_cte(self):
        return self.execute()

class CustomDialect(Dialect):
    class Tokenizer(Tokenizer):
        KEYWORDS = {
            **Tokenizer.KEYWORDS,
            "CLUSTER": TokenType.FUNCTION,
            "SCATTER": TokenType.FUNCTION,
        }

    class Parser(Parser):
        FUNCTIONS = {
            **Parser.FUNCTIONS,
            "CLUSTER": Cluster.from_arg_list,
            "SCATTER": Scatterplot.from_arg_list,
        }

    class Generator(Generator):
        TRANSFORMS = {
            **Generator.TRANSFORMS,
            Cluster: lambda self, expr: f"CLUSTER({self.sql(expr.this)})",
            Scatterplot: lambda self, expr: f"SCATTER({self.sql(expr.this)})",
        }

df = pd.DataFrame({
    "id": range(1, 10),
    "price": [10, 20, 30, 200, 220, 250, 1000, 1100, 1200],
})

query = """
WITH resultados_cluster AS (SELECT CLUSTER(price) FROM df)  
SELECT SCATTER(price) FROM resultados_cluster;
"""

# Parseando a query com o dialeto customizado
# expression_list = sqlglot.parse(query, dialect=CustomDialect)

# for expr in expression_list:
#     if isinstance(expr, Cluster):
#         expr.execute(df)

# Parseando a query
expression = sqlglot.parse_one(query, dialect=CustomDialect)

# Encontrar a expressão CLUSTER na AST
cluster_expr = expression.find(Cluster)
print(cluster_expr)

# Executar a função CLUSTER e gerar um DataFrame com os resultados
# df_clusterizado = cluster_expr.to_cte(df)

# vis_exp = expression.find(Scatterplot)
# vis_exp.to_cte(df)

# from visualizations import Visualization

# vis = Visualization()

# vis.scatter(df_clusterizado['id'],df_clusterizado['cluster'])

# Exibir o DataFrame final com os clusters
# print(df_clusterizado)



None
